#VacationPy
Maria Barrera -- 02/07/2021

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# data file -- created from mainWeatherPy
city_path = "output_data/cities_file.csv"
city_path

'output_data/cities_file.csv'

In [3]:
# Read the city data 
city_data = pd.read_csv(city_path)
city_data

City ID           City  Cloudiness Country        Date  Humidity  \
0       8512          ancud           0      CL  1612764503        94   
1       8487   punta arenas          90      CL  1612764504        44   
2       8074       katsuura          75      JP  1612764505        40   
3       4900  saint anthony          90      US  1612764506        71   
4       7271         avarua          50      CK  1612764507        78   
..       ...            ...         ...     ...         ...       ...   
272     1450      stornoway          75      GB  1612764751        69   
273      656         killam         100      CA  1612764755        75   
274     7411       buraydah          75      SA  1612764756        94   
275     9486     gold coast          75      AU  1612764757        61   
276     6900     agucadoura          40      PT  1612764757        82   

         Lat       Lng  Max Temp  Wind Speed  
0   -41.8697  -73.8203    287.59        3.09  
1   -53.1500  -70.9167    288.15        3.09  
2    35.1333  140.3000    282.15        2.57  
3    45.0205  -93.2180    254.15        3.09  
4   -21.2078 -159.7750    301.15        3.60  
..       ...       ...       ...         ...  
272  58.2093   -6.3865    274.15        5.66  
273  52.7834 -111.8518    238.15        2.57  
274  26.3260   43.9750    285.15        3.09  
275 -28.0000  153.4333    301.48        9.26  
276  41.4318   -8.7804    284.15        3.60  

[277 rows x 10 columns]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Convert humidity as a list
humidity_list = city_data['Humidity']
humidity_list

0      94
1      44
2      40
3      71
4      78
       ..
272    69
273    75
274    94
275    61
276    82
Name: Humidity, Length: 277, dtype: int64

In [5]:
# Create a map using city coordinates to set markers
marker_locations = city_data[['Lat', 'Lng']]

# Create a marker_layer using humidity list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Humidity: {humidity}" for humidity in humidity_list])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#for reference only
city_data.columns

Index(['City ID', 'City', 'Cloudiness', 'Country', 'Date', 'Humidity', 'Lat',
       'Lng', 'Max Temp', 'Wind Speed'],
      dtype='object')

In [7]:
#  temperature conversion & column update

new_df = city_data

# for testing only ---------------
# maxtemp = 294.26
# temp_f = (maxtemp - 273.15) * 9/5 +32
# temp_f
# --------------------------------

for ind in new_df.index: 
    maxtemp = new_df['Max Temp'][ind]
    temp_f = (maxtemp - 273.15) * 9/5 +32
    new_df['Max Temp'][ind] = temp_f
    
    # for testing only
    #print(temp_f)
    #print(new_df['Max Temp'][ind], ind) 
    
new_df.head()

<ipython-input-7-51d7cc9f05bd>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Max Temp'][ind] = temp_f


City ID           City  Cloudiness Country        Date  Humidity      Lat  \
0     8512          ancud           0      CL  1612764503        94 -41.8697   
1     8487   punta arenas          90      CL  1612764504        44 -53.1500   
2     8074       katsuura          75      JP  1612764505        40  35.1333   
3     4900  saint anthony          90      US  1612764506        71  45.0205   
4     7271         avarua          50      CK  1612764507        78 -21.2078   

        Lng  Max Temp  Wind Speed  
0  -73.8203    57.992        3.09  
1  -70.9167    59.000        3.09  
2  140.3000    48.200        2.57  
3  -93.2180    -2.200        3.09  
4 -159.7750    82.400        3.60

In [8]:
#vacation selection criteria
#temperature 70-80 degrees
#wind speed <10
#cloudiness = 0

subset_df = new_df[(new_df["Max Temp"] >= 70)]
subset_df = subset_df[subset_df["Max Temp"] <= 80]
subset_df = subset_df[subset_df["Wind Speed"] < 10]
subset_df = subset_df[subset_df["Cloudiness"] == 0]
subset_df

City ID             City  Cloudiness Country        Date  Humidity  \
14   2005410         pochutla           0      MX  1612764460        88   
94      8584  puerto colombia           0      CO  1612764496        83   
99   2000096         santa fe           0      AR  1612764552        79   
193     8682             lima           0      PE  1612764540        88   
208  2008823           parana           0      AR  1612764677        81   
232     7545          sharjah           0      AE  1612764637        56   
253     9070        porbandar           0      IN  1612764729        24   

         Lat      Lng  Max Temp  Wind Speed  
14   15.7432 -96.4661    73.004        1.44  
94   10.9878 -74.9547    78.800        4.63  
99  -31.6333 -60.7000    73.994        0.81  
193 -12.0432 -77.0282    73.004        4.63  
208 -31.7320 -60.5238    73.994        2.26  
232  25.3573  55.4033    75.200        2.06  
253  21.6422  69.6093    78.800        5.14

In [9]:
# Drop any rows will null values
subset_df = subset_df.dropna(how="any")
subset_df

City ID             City  Cloudiness Country        Date  Humidity  \
14   2005410         pochutla           0      MX  1612764460        88   
94      8584  puerto colombia           0      CO  1612764496        83   
99   2000096         santa fe           0      AR  1612764552        79   
193     8682             lima           0      PE  1612764540        88   
208  2008823           parana           0      AR  1612764677        81   
232     7545          sharjah           0      AE  1612764637        56   
253     9070        porbandar           0      IN  1612764729        24   

         Lat      Lng  Max Temp  Wind Speed  
14   15.7432 -96.4661    73.004        1.44  
94   10.9878 -74.9547    78.800        4.63  
99  -31.6333 -60.7000    73.994        0.81  
193 -12.0432 -77.0282    73.004        4.63  
208 -31.7320 -60.5238    73.994        2.26  
232  25.3573  55.4033    75.200        2.06  
253  21.6422  69.6093    78.800        5.14

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
# set up hotel df to hold information
hotel_df = pd.DataFrame()

hotel_df['Hotel Name'] = ""
hotel_df

Empty DataFrame
Columns: [Hotel Name]
Index: []

In [ ]:

#### find the closest hotel to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "39.952583,-75.16522",  # philadelphia coords
    "rankby": "distance",
    "type": "hotel",
    "key": gkey,
}
# use iterrows to iterate through pandas dataframe
for index, row in types_df.iterrows():

    # get restaurant type from df
    restr_type = row['ethnicity']

    # add keyword to params dict
    params['keyword'] = restr_type

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {restr_type}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {restr_type} restaurant is {results[0]['name']}.")
        
        types_df.loc[index, 'name'] = results[0]['name']
        types_df.loc[index, 'address'] = results[0]['vicinity']
        types_df.loc[index, 'price_level'] = results[0]['price_level']
        types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# for reference only

#### find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "39.952583,-75.16522",  # philadelphia coords
    "rankby": "distance",
    "type": "restaurant",
    "key": gkey,
}
# use iterrows to iterate through pandas dataframe
for index, row in types_df.iterrows():

    # get restaurant type from df
    restr_type = row['ethnicity']

    # add keyword to params dict
    params['keyword'] = restr_type

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {restr_type}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {restr_type} restaurant is {results[0]['name']}.")
        
        types_df.loc[index, 'name'] = results[0]['name']
        types_df.loc[index, 'address'] = results[0]['vicinity']
        types_df.loc[index, 'price_level'] = results[0]['price_level']
        types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
